In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from datetime import datetime
import pandas as pd
import ast

# .env ファイルの読み込み
load_dotenv()

# APIキーの取得
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("環境変数 'OPENAI_API_KEY' が設定されていません。")

# OpenAIクライアントの初期化
client = OpenAI(api_key=api_key)

def format_debate_transcripts(content):
    """
    議論のトランスクリプトをフォーマットする関数
    
    Args:
        content: スピーチ内容のリスト
    
    Returns:
        フォーマットされたトランスクリプト文字列
    """
    if len(content) == 8:
        transcripts = f"""## Government 1st
{content[0]}

## Opposition 1st
{content[1]}

## Government 2nd
{content[2]}

## Opposition 2nd
{content[3]}

## Government 3rd
{content[4]}

## Opposition 3rd
{content[5]}

## Opposition 4th
{content[6]}

## Government 4th
{content[7]}"""
    elif len(content) == 6:
        transcripts = f"""## Government 1st
{content[0]}

## Opposition 1st
{content[1]}

## Government 2nd
{content[2]}

## Opposition 2nd
{content[3]}

## Opposition 3rd
{content[4]}

## Government 3rd
{content[5]}"""
    else:
        raise ValueError(f"Unexpected content length: {len(content)}")
    
    return transcripts

def create_prompt(title, content):
    """
    評価用のプロンプトを作成する関数
    
    Args:
        title: モーションのタイトル
        content: スピーチ内容のリスト
    
    Returns:
        完成したプロンプト文字列
    """
    transcripts = format_debate_transcripts(content)
    
    prompt = f"""# Instruction
Evaluate the argumentative interaction in debate. In this context, argumentative interaction refers to how effectively debaters engage with, respond to, and build upon opposing viewpoints rather than dismissing or overlooking them.

You will evaluate the following statement: "Both teams demonstrated effective argumentative interaction by consistently engaging with, responding to, and building upon each other's arguments throughout the debate."

Rate your level of agreement with this statement using the scale provided below.

## Rating Scale:
- **Agree**: Both teams consistently referenced and responded to each other's actual arguments
- **Rather agree**: Both teams addressed most major opposing points with some engagement on minor details
- **Rather disagree**: Both teams addressed some opposing points, with at least one team missing several key arguments in all issues
- **Disagree**: Teams showed limited engagement with what opponents actually argued, with at least one team making arguments largely running in parallel

## What Counts as Real Argumentative Interaction:
- **Direct references**: "The opposition said X, but we think Y because..."
- **Specific engagement**: "Regarding their point about X..."
- **Direct refutation**: Challenging opponent's logic or evidence
- **Building/extending**: Developing or countering opponent's arguments

## What does NOT count as interaction:
- **Topic overlap without engagement**: Discussing same subject without referencing opponent
- **Parallel arguments**: Making independent points without addressing opposing claims
- **Generic responses**: Broad statements not tied to specific opposing arguments

## What to EXCLUDE from evaluation:
- **Argument quality**: How good or persuasive the arguments were
- **Missed opportunities**: What teams should/could have argued
- **Potential improvements**: How rebuttals could have been better
- **Alternative strategies**: Whether different approaches would be superior
- **Evidence quality**: Strength of reasoning or supporting material

# Information
The motion for this round was: {title}

# Debate Transcripts
{transcripts}

# Output Format
Rating: [Select one: Agree / Rather agree / Rather disagree / Disagree]

Reasoning:
[List specific examples of teams responding/not responding to actual opposing arguments. Do not suggest improvements or evaluate argument quality.]"""
    
    return prompt

# ログディレクトリの作成（存在しない場合）
os.makedirs('logs', exist_ok=True)

# CSVファイルを読み込み
df = pd.read_csv('motion_title_content_english.csv')

for index, row in df.iterrows():
    id = row['id']
    title = row['Title']
    content = ast.literal_eval(row['Content'])
    
    # プロンプトの作成
    prompt = create_prompt(title, content)
    
    # OpenAI APIの呼び出し
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user", 
                "content": prompt
            }
        ]
    )
    
    # 応答の表示
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-4]
    log_filename = f"logs/log_{id}_{title}_{current_time.replace(':', '-').replace(' ', '_').replace('.','-')}.txt"
    
    with open(log_filename, "a", encoding="utf-8") as log_file:
        log_file.write(f"Timestamp: {current_time}\n")
        log_file.write(f"Response detail: {response}\n")
        log_file.write(f"---------------Response Content----------------\n{response.choices[0].message.content}\n")
        log_file.write(f"------------------Input Prompt---------------\n{prompt}\n-------------------\n")
    
    # 結果の表示
    print(f"ID: {id}, Title: {title}")
    print(response.choices[0].message.content)
    print("-" * 80)

ID: 1, Title: The Kansai 2018 R4
Rating: Rather agree

Reasoning:

1. **Direct Engagement and Rebuttal:**
   - The Opposition 1st speaker directly engages with the Government's argument about the dominant narrative incentivizing terrorists to attack women by providing specific examples of other types of attacks that gain media attention (e.g., suicidal attacks, indiscriminate killing).
   - The Government 2nd speaker rebuts the Opposition's claim that women are inherently more vulnerable during conflicts by arguing that showing men's suffering can also evoke a strong reaction and attract media attention.

2. **Specific References to Opponent's Points:**
   - The Government 1st speaker addresses Opposition's theoretical assumptions about the necessity of a dominant narrative to gain attention by mentioning specific cases where brutal imagery of male victims could be equally effective.
   - The Opposition 2nd and 4th speakers highlight and respond to the Government's point about post-con

## Old version

In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from datetime import datetime
import pandas as pd
import ast

# .env ファイルの読み込み
load_dotenv()

# APIキーの取得
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("環境変数 'OPENAI_API_KEY' が設定されていません。")

# OpenAIクライアントの初期化
client = OpenAI(api_key=api_key)

# CSVファイルを読み込み
df = pd.read_csv('motion_title_content_english.csv')

for index, row in df.iterrows():
    id= row['id']
    title = row['Title']
    content = ast.literal_eval(row['Content'])
    if len(content) == 8:
        prompt = f"""
# Instruction
Please evaluate the following parliamentary debate round based on the degree of engagement of arguments. Return the score from 1 to 4. Then, explain the reason for the score in detail.

# Information
The motion for this round was: {title}

# Debate Transcripts
## Government 1st
{content[0]}

## Opposition 1st
{content[1]}

## Government 2nd
{content[2]}

## Opposition 2nd
{content[3]}

## Government 3rd
{content[4]}

## Opposition 3rd
{content[5]}

## Opposition 4th
{content[6]}

## Government 4th
{content[7]}
"""
    elif len(content) == 6:
        prompt = f"""
# Instruction
Please evaluate the following parliamentary debate round based on the degree of engagement of arguments. Return the score from 1 to 4. Then, explain the reason for the score in detail.

# Information
The motion for this round was: {title}

# Debate Transcripts
## Government 1st
{content[0]}

## Opposition 1st
{content[1]}

## Government 2nd
{content[2]}

## Opposition 2nd
{content[3]}

## Opposition 3rd
{content[4]}

## Government 3rd
{content[5]}
"""

    # reasoningモデル
    response = client.responses.create(
        model="gpt-4o",
        # reasoning={"effort": "high"},
        input=[
            {
                "role": "user", 
                "content": prompt
            }
        ]
    )

    # 応答の表示
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-4]
    log_filename = f"logs/log_{id}_{title}_{current_time.replace(':', '-').replace(' ', '_').replace('.','-')}.txt"
    with open(log_filename, "a", encoding="utf-8") as log_file:
        log_file.write(f"Timestamp: {current_time}\n")
        log_file.write(f"Response detail: {response}\n")
        log_file.write(f"---------------Reasoning Summary----------------\n{response.output_text}\n")
        log_file.write(f"------------------Input Prompt---------------\n{prompt}\n-------------------\n")

    # print(response)
    print(response.output_text)

## 1試合文のプロンプトべた書きバージョン

In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from datetime import datetime

# .env ファイルの読み込み
load_dotenv()

# APIキーの取得
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("環境変数 'OPENAI_API_KEY' が設定されていません。")

# OpenAIクライアントの初期化
client = OpenAI(api_key=api_key)

prompt = """
# Instruction
Please evaluate the following parliamentary debate round based on the degree of engagement of arguments. Return the score from 1 to 4. Then, explain the reason for the score in detail.

# Information
The motion for this round was "This House regrets the dominant narrative of women as non-violent and vulnerable in conflicts and humanitarian crises."

# Debate Transcripts
## Government 1st
Intention of the Western media, like revealing this dominant narrative regarding the vulnerable women, they don't have the malicious intention, but they rather have good intention in order to save women, save children.
But the outcome is different, Mr. Speaker.
Because they target, these women are vulnerable, they target these children are so vulnerable that they are weak, they are non-violent.
That leads to the bad outcome that ultimately these women and children are being targeted by the terrorists, are being targeted by those post-conflict sexism.
We believe that is something horrible, as the Western media has done.
We believe, we beg to propose.

I'm going to talk about mainly three things.
Firstly, how this dominant narrative badly influences post-conflict zone sexism.
Secondly, how this dominant narrative is badly affecting these women and children regarding these kinds of terrorist attacks.
Thirdly, and lastly, how this dominant narrative leads to unconscious sexism.

First thing first, about the dominant narratives.
What we mean by dominant narrative is those, mainly those Western medias, like picturizing these women and children who are being raped, who are being brutally murdered in the conflicts or the humanitarian crisis zones.
They picturize those women and children being forced by the power of male, and being forced by the power that they are being raped and they are being weak toward those kinds of attacks.
They have the massive incentive to use women and children as a vulnerable and non-violent actor.
It's because it's pretty shocking that they are using the picture of women being raped, picture of children being stabbed by the knives or those kinds of things.
Because those kinds of shocking pictures and shocking words are going to raise the viewing rate or those kinds of magazines or those kinds of things.
Therefore, those Western media has a massive incentive to make those kinds of narratives very dominant.

As a consequence, by this kind of dominant narrative done by Western medias, there will be much more attention toward female victims than the male victims, Mr. Speaker.
For example, in Boko Haram, for example, in Syria, for example, in ISIS areas.
As a consequence, what happens is that there will be a massive amount of discussion inside of U.S. councils or WTOs or those kinds of more policies sending peacekeeping operations or humanitarian aid toward those kinds of women and children who are dying on the ground inside those kinds of conflict zones.
What happens is that peacekeeping operations and humanitarian aid enter conflict zones and prioritize women and children than the men.

What happens is that more male victims are going to increase rather than those kinds of female victims, Mr. Speaker.
This leads to the post-conflict sexism, Mr. Speaker.
What do I mean by this?
It means that those kinds of male victims who are brutally killed or male victims who are brutally injured by those kinds of war operations, they feel that they need a reward in a post-conflict society.
They need some kind of class.
They need some kind of money.
They need some kind of reward that they have done in conflict zones.
Therefore, they cannot give away their money or their tax revenues toward women because they think that they were already saved by the Western society.
They were already prioritized than our lives in the conflict zones.

We believe in that kind of scenario, that leads to the post-conflict sexism, Mr. Speaker, that those men are getting a reward and women are getting nothing in return because they have already got those kinds of reward or the treatment during those kinds of conflict zones.
We believe, rather, that saving those kinds of women and men equally leads to the betterment of these kinds of situations because those kinds of male victims cannot make the excuse that they were de-prioritized than female victims and therefore they cannot use those kinds of rhetoric.
We believe this kind of sexism is going to better off on our side of the house.

Secondly, Mr. Speaker, those kinds of dominant narratives lead to more targeting women and children by the terrorist attacks.
Why is that?
It's because those kinds of terrorists, for example, ISIS, knows that Western media uses female and children as a news title or those kinds of eccentric news reports.
Therefore, they have a massive incentive to spread their political messages throughout those kinds of news outlets, which means that by targeting women and children, they will have a massive potential or possibility to make their political messages throughout those kinds of Western media.
Because they know that Western media have an incentive to use and target those female victims' reports or the children victims' reports.

Therefore, what those kinds of Western media is doing is the opposite of their aim, Mr. Speaker.
They are trying to target those women and children and prioritize their lives, but what they are doing is the opposite.
They are rather giving those kinds of opportunities for those terrorist groups to attack these women and children.
We believe that is morally horrible.

Thirdly and lastly, why this dominant narrative leads to unconscious sexism.
Dominant narrative regarding women who are pictured as weak and non-violent actors is like 100,000 women who were brutally killed and raped.
They were forced by men's power.
They were overly done by Western media and that leads to unconscious sexism.

So why is this bit of more sexism more important than the increased amount of aid or aid going to people who actually need it?

Because, Mr. Speaker, first of all, we want to question why do you prioritize women's lives over the male, right?
We could see why that is justified in the first place, but second of all, we do not want, by equalizing those kinds of aids and treatments, we believe that the post-conflict society is going to be better off, right, better off, Mr. Speaker, because sexism is something really rooted in the society and that is going to give less rights to women, Mr. Speaker.
We believe that is important as well.

So by picturizing women as weak and non-violent actors, that is going to make people unconsciously sexist toward women.
They are going to say that because you are weak, you cannot wear slutty clothes, you are going to be raped in the town, you cannot wear those kinds of slutty clothes because you are weak, right?
Those kinds of narratives are going to be dominant and that is going to lessen women's rights and freedom.

## Opposition 1st
Okay, two issues from leader position.
Firstly, I will talk about the realistic characterization of women in conflict.
We do recognize that some women are strong, who are capable enough to fight back against men.
But unfortunate reality is that in general, women are likely to be comparatively more vulnerable than men,
and comparatively more likely to target the atrocious consequences.
Thus, they need more support and protection.

Secondly, I will talk about why, due to the dominant narrative,
they uniquely make it possible for us and surroundings or NGOs or donors
to provide the necessary support and a decent amount of support
so that their women are going to be protected.

Before that, I will present the responses to the Prime Minister.
Firstly, they said that this dominant narrative incentivized terrorists to exclusively target women in the conflict areas.
Firstly, we do concede that the status quo is not perfect,
but we think that due to this narrative, the situation is currently going to be changed.
For example, the International Criminal Court actually introduced the rape or sexual assault as crimes against humanity,
and the international recognition that sexual assault requires special recognition.
And also, there is a certain international criminal bashing towards the ignorance of the government
who has an incompatibility to deal with sexual assault.

But furthermore, when it comes to the media's emphasis on gaining attention,
we see that terrorist groups are also already relying on the other kind of things.
For example, they are relying on the suicidal attack, or they are relying on the indiscriminate killing,
and this is even more attracting when it comes to getting attention through the media.
So I don't think this is a core motive for terrorist groups to exclusively target women on the ground.

They said that male protection is going to be on the mind, compared to the female.
But as I explained earlier, we think that given that women are likely to need special protection,
we think it's proportionate and we think it's good.

Lastly, they said that this dominant narrative is leading to the post-conflict system.
But we think that if those kinds of leaders or men in society have the intention to discriminate women,
we think that they are going to anyway find any excuse to discriminate women or let's reward women on the ground.
Maybe, for example, they may say that due to their pregnancy, women cannot, unfortunately,
work exactly the same as men or so forth.
Anyway, sexists are going to find that kind of excuse and discriminate women.

But furthermore, even in that case, we have to be comparative.
While it's also a kind of social position of the post-conflict society,
it's kind of more important than the immediate help of the women in the conflict zone.
We think it's more important.

So I'll just talk about the realistic characterization of the women in the conflict.
So in general, women are fortunately likely to be comparatively more valuable than men for several reasons.
Firstly, physically, women are kind of likely to be weaker or less physically capable to fight back exactly the same level as men.
It's just that women are likely to be the target of the conflict.
But furthermore, we believe that women are exclusively likely to be the target of sexual assault on the ground.
We do recognize that men also can be the victim of sexual assault.
But we think the rate is disproportionately higher on the male and female.
If we look at the reality, for example, we come to the abduction by the terrorist group like Kohara.

Most of the victims, the abductors, are teenage girls in the school, and they are abducted by the terrorist group.
So we think in the realistic context, those who are an individual, women are more likely to be the target.
We think they need special protection.
But furthermore, maybe the need for sanitary protection due to the biological feature.
Maybe, for example, there is a pregnant mother who needs much more protection, who is more vulnerable, etc.
So we think that realistically, women are likely to be relatively more vulnerable, thus they need special protection and support.

So given the nature, let's look at the second issue about why the dominant narrative is necessary in order to protect these women on the ground.
So we think, firstly, that the dominant narrative allows the point of peacekeeping operation, intervening military or any other individual, etc.
to provide more support to the women on the ground.
Maybe, for example, we come to this shelter, which has a lot of women.
Maybe we can deploy more soldiers so that we are able to protect them more.
Or, for example, we are giving extra support, medical support or sanitary support to pregnant women, etc.
And also extra support is going to be available on our side of the house.
And also, as I already told you, ICC's recognition of rape or sexual assault as a crime against humanity
are also introduced due to this dominant narrative that recognizes that women are relatively more vulnerable on the ground.
And we think it's beneficial.

Secondly, we think that this dominant narrative allows the...
What is the connection between ICC recognizing sexual rape as assault and making people believe that women are non-violent?
This is because, due to the dominant narrative, people in the liberal democracy can't stand women are more likely to be victimized than any other assault in Korea, etc. No thank you.
So we think, yeah, that's really important to say that.

But secondly, we think that due to this dominant narrative, it kind of enables the individual donors in the liberal democracy or the advanced nation
to think about the women, the situation of the women on the ground, and think about what kind of support they are going to need, etc.
And we think it's critically important, right?
Because the majority of the average citizen in the liberal democracy haven't experienced the situation of the conflict that much,
and it's kind of hard for individuals to imagine what kind of support they are going to need, etc.

And we think that the dominant narrative allows those average citizens or individuals who can be the potential donors to the NGO
or the supporter towards the women on the ground, right?

And they are going to think that, you know, that women may need, for example, certain special, like, sanitary protection,
or maybe can't, like, want to need extra protection, extra, like, deployment to the military due to the high tendency of sexual assault.
And we think that it's also the recognition of what, like, understanding of the exact necessity of support is very important, right?
Because in order to, like, maximize the utility or, like, benefit from the support and the donation,
we think that this, like, dominant narrative is critically important.

So overall, we think that, you know, given that it's, like, unfortunately that women are more likely to be vulnerable and targeted at a special time,
we think that it's, like, beneficial and reasonable to use the dominant narrative to gather support and help them.

## Government 2nd
The opposite side claims that without this dominant narrative that women are non-violent in conflict zones.
We are not able to attract the media's attention, the western's attention, and the NGOs in the west are not going to be able to protect the people who are suffering in conflict zones.
But we reject this analysis.
We think that there's a lot of people who are suffering in those conflict zones who are not women.

For example, maybe men's child therapy, their hands, because of this really aggressive guerrilla groups.
We think those kind of images can actually shock people and feel that there's a necessity to actually reach out and protect them.
We don't think that that dominant narrative of women suffering is something that's necessary.
There's a lot of children who are suffering, men suffering, and also women.

We think that we can't have women suffering, but we just can't have all kinds of people just show that all kinds of people are suffering.
We think that in itself is something that is not.
We think that this kind of image that we are creating, that women are weak, women are vulnerable, actually makes women unable to actually fight back.
Not only during the conflict zone, not only during the conflict zone, but also after the conflict zone.

And we think we are making the women in a weaker position, so that is why we are very happy to propose this motion.
So I'm going to maybe talk about two things.
First, I'm going to talk about why it is still going to go to those kind of conflict zones.
And secondly, I'm going to talk about which is better for the female.

But before moving on to my points, I have a refutation to their side of the house.
So the leader of the option refuted to my partner that men will always make an excuse to actually exploit women.
But we think that may be true, but if there's less excuses, that means that there are less reasons that they can justify their actions.

So we think that if there are less excuses, for example, women can fight back and say that that is not right because I also contributed to the society.
I also fought back.
I also contributed during the conflict zone, so I should have the right to decide on my political will.
I should have the right to decide to go outside because I can actually protect myself.

So first, I want to talk about why aid is still going to go in those kind of conflicts.
So even if there's not a dominant narrative, that women are not violent and vulnerable.
So we were talking about like, so they talked about we have to show images that women are getting raped, that women are getting, like for example, there's need to dominate narratives of women being harmed and raped or those kind of things.

But we think that in places where in conflict zone, we think that not only women are suffering of getting raped.
For example, in places like in Congo, there are many children and many also men getting killed brutally.
Maybe they're cut their limbs because they think that cutting limbs would actually make their opponents weaker.
And we think that those kind of brutal images can be shown even in our side of the house.

And we think that if media can find other images that are very shocking, they have an incentive to do it.
So we think that even if women are not being raped and they don't show a dominant narrative of women being raped, we think that it can also attract the media's attention if some people are actually suffering.
And we think that because there's an overt dominant narrative that women are getting, like for example, suffering, we think that the amount of resources used to save the men or other people are not going to be used.
And we think at the end of the day, we think that the amount of people that can be saved are going to be the same.

We think on our side, maybe fewer women are saved.
We can see to that.
But at least maybe more men are going to be saved because of this less dominant narrative.
And we think that in itself is not really a bad thing.
And we don't understand why they want to prioritize women over men.
We think all lives are equal.
And if men are maybe more protected, we think that in itself is not really a bad thing.
And we think there are a lot of men who are suffering.
And we think at the end of the day, the number of people who are going to be saved is the same.

Secondly, I want to talk about which is better for the female at the end of the day.
So firstly, about during the conflicts.
So we think that because we actually internalize this, because NGOs are in the West, actually prioritize protecting women.
We think women have less capacity to actually protect themselves.

What do I mean by this?
Because there is no necessity for women to actually maybe arm themselves because the West is there to protect them.
And because the West thinks that you have to be protected.
We think that is why women themselves do not arm themselves.
They said that women are vulnerable.
So that is why women are not able to protect themselves.

But we think this narrative in itself actually makes women vulnerable.
We think women in itself can actually protect themselves.
So if you have a gun, even if your man is much more masculine and has much more physical strength, you can kill that another person.
So we think that physical strength in itself is not a matter in this debate.

So we think that those women can arm themselves.
And when we feel that they are capable to arm themselves, they won't be able to do it.
Because this dominant narrative is so strong, they are not able to arm themselves.
They think that they are vulnerable.
They lose hope and protect themselves.
And that actually leads the women to be targets of, for example, a lot of terrorist groups.
Or they are getting targets of getting raped.
We think this narrative in itself makes women weak.

 Are you saying if this narrative doesn't exist, can you make women suddenly take guns and fight against men and protect themselves?

I'm talking about being comparative.
I don't think this is an absolute solution.
But we have to be comparative.
That at least women will feel that, oh, I have to protect myself on my own.
And men will fear that, oh, if I actually attack this woman, maybe I might get killed.
We think that people's thinking is going to change.
And we think that women are going to be less targets.
And we also think that there are going to be fewer targets of, for example, Boko Haram or ISIS.
And we think they never actually refused to do that.

And secondly, after conflicts.
I'm going to talk about after conflicts.
We think that in the status quo, women are seen as vulnerable individuals.
And they are seen as people who did not contribute during the conflict zone.
Maybe they did house chores.
Or maybe they supported it.
But they are seen as secondary individuals, second-class citizens.
Because there was less contribution.
And because we are seeing that these people are vulnerable and they did not contribute, we are making society can create a norm.
The woman is less contributing, so let's get them fewer rights.

And we are also sending a message.
Being supportive, for example, of the people who are fighting.
Or doing house chores is also not a good contribution.
It's not something that is a significant contribution.
And that makes women much more vulnerable after conflicts.
And we think that in itself is bad for the women.

So that's why we are very happy.

## Opposition 2nd
Okay.I'll bring you two new substantives.
First, why the use of a vivid and vulnerable activity to increase attention and material support to areas.
Why the piling of the aid and support going to the areas is not the same in two paradigms but it's increased on our side of the house because we have more vulnerable, clear images to the public where people feel more need and sympathy to support these areas.
Second, I'll talk about why this dominance of a narrative is important.
Why people should be aware of the image of what women actually need.
Why that pre-imposed exposure to the idea is necessary for people to, in that chaotic, immediate need to help people, they are able to understand what donation is necessary, what they should act right now.
For inexperienced NGOs who go into the area, knowing about the situation is so important.

Before I go on, a couple of independent rebuttals.
So, first, they talk about how, you know, it leads to some sort of sexism because, you know, they're vulnerable or something like that from the Prime Minister.
They told us what sort of sexism and action it actually leads to and I think the logic is very blurred.
And also, we have to recognize that, you know, in these areas, or it's universally, helping women and children first as a man is a pride of a man.

It's seen as a responsibility of a man.
Also, in any form of conservative religion, it's seen that it's the responsibility of the man to protect women and children first.
That also adds to the fact that they won't see that they are deprioritized or they are being abandoned.
They see it as a pride that women are actually given more support.
They will actually facilitate to let women go on with it.
So, it's wrong for them to say that men are so greedy and go on with it.
We think that those men do exist, but they're a minor part of society.
I think that's a bizarre characterization.

But also, even if we concede that sexism might happen to a certain scenario, we think that's far less important than people being saved by the fact that they're given sanitary goods.
They're saved by the fact that they're given food and clothing or protection from, I think, mosquitoes and things like that.
So, it's only possible because they have a substantial amount of aid provided to these people.
And we think that, talking about the larger individuals in that vulnerable situation, this bit of blurred sexism is far less important than that.

Then, they also talk about, from FPM and BPM, that they are seen as vulnerable, so terrorists are likely to more attack them.
Or men, I like to say that they're weak, so you should obey us or use it as a rhetoric for sexism and things like that.
Firstly, I think Errol talked about how terrorists don't really care about it, which was not responded.
But further, there are plenty of other reasons for terrorist organizations or these men to be sexist on women.

Firstly, there are reasons to say that, you know, for terrorists, they are intuitively weak in the physical sense.
They may have greed for sexual desire, so there are other reasons to attack women first, unfortunately, as men, male soldiers.
Or they simply think that they have the extremist views of, for example, Islamic beliefs that say that they should subordinate women, so they should attack first instead of men.
Or they might have this strategic interest attacking women groups, this historical know-how as tribes and things like that.
And given the fact that terrorists often don't really care about it, it's that this additional one reason of a narrative doesn't really change their calculus because they already have tens of other reasons to do that.

If the tens of other reasons don't change their calculus, I don't understand why this bit of a narrative that these terrorist people haven't been exposed to, maybe, has any influence to it.
And even if not, because even if you add it, because there's not much of a difference, the substantial difference is really unclear.
Even if you make male, like, trying to make it obedient, you know, this sexist belief or preferences of men to go on with it already exists.
This bit of narrative that exists in the context also has no clear...
Sorry.
It amplifies that narrative, but it's not clear to what extent that's important.

No, thank you.
But also importantly, this narrative, as they set up, is a broadcasting of Western media.
It's not about how the narrative forms in the conflict zone, but rather about people around the area.
Not in the area, but around the world.
And we also think that also proves the fact that the influence they want to suggest is not that huge.
Sure.


So, why have you told us that these conservative sexism are quite minor, limited to not minor people?
Now, you're saying that there are lots of conservative sexism does exist, so women won't be able to get out of it.
Which is it?
There are a lot more conservatives or less conservatives?

I'm saying that if they do exist, it's that it's not going to be rid of a big issue.
So, no, we're not saying that they're a majority.
I don't really get it.
Connection Between ICC and Dominant Narrative
Then there's a bit about ICC and things like that.
We understand that there's not clear connection between ICC's recognition of rape and vulnerability of women.
The thing is that it was necessary for the discussion and investigation of female issues even at core, that we needed social pressure to have women being protected.
Because in the past, the PKOs, the reporting, didn't even have a section for gender or women.
We didn't really investigate the rape.
It was all just about protecting against guerrillas and things like that.

Why is the use of vivid images so important?
Importantly, the support to these areas are vital to the survival of people on the ground.
It might be aid, it might be interventions, it might be sanctions, but all of this is contingent on the fact that you have a necessary critical mass of people who care about the area, who are angered by the situation and want to support these areas.
The more support and attention you have, you have more people saved, or quickly saved because of interventions that were done a bit more quickly.

Obviously, the sympathy that is created by the image of the media is enhanced when you use, unfortunately, more vulnerable looking actors.
The use of the image of a woman is far more vulnerable looking than a man.
Also, religious narratives and social narratives about how men should be obliged to protect women also enhance the fact that when women are put in vulnerable positions compared to men, it provokes some emotional feelings that they should protect them compared to men.
They do have the tool of children in their side of the house too, but the mixed use of women would also lead to an accelerated power to have more attention in these areas, and an increased amount of power means that we are able to save these people on the ground more, and that means that more aid is coming around the side of the house.

So they say that men are going to receive less aid, but if the pie is increased, they don't receive less.
But also, even if we can see that men don't receive sufficient aid, it's that men have more energy and have vitality compared to women.
So being prioritized in the situation of aid is far less important than women being provided of the actual needs on the ground.
Shortly on why dominant narratives.

Unfortunately, people, when they want to actually help a certain area, they just delay the things that they actually think they need.
But the clothing and the food that they actually provide might not be a necessary thing, it might be a burden to the actual area.
To prevent that, it's important that people expose the image of what actually women need.
The image that women are suffering in this particular way through the media, providing the understanding that the issues are real, the issues are sanitary conditions, you need napkins instead of actual random foods or clothing and things like that.

That's only possible when the dominant narrative is also public and people are pre-emptively aware of the particular situation.
Inexperienced NGOs also will be able to be aware of the situation and not provide random unnecessary things but actual needy things for women.
That's only possible if we can have a narrative and not an alternative systematic approach.
Thank you.

## Proposition 3rd
This debate is not about whether we are allowed to put this picture as, for example, MSF, like for example Red Cross, showing a picture that women are being brutalized in African countries.
This debate is about whether that should be dominant, whether that narrative should be dominant, that everyone believes that it is always a female who are being brutalized.

It is always a female who are never going to be violent, who are never going to be non-vulnerable.
Women are always weak, they cannot do anything by themselves, and that narrative should be dominated within a society and make every person in a society, including the white men in the United States or the female in Africa, to believe that women should be non-violent.
We do not think so because that obviously does not reflect the reality, and we think that incentivizes multiple kinds of actors to believe also that women are non-violent and vulnerable, they won't fight back.

We can utilize and we can exploit the weakness which has been created by the Western media to harm women.
We think saying that women are non-violent and vulnerable does not protect women, they are going to be more targeted, they are going to be exploited of the weakness which has been created by media, we think that is wrong, we should go against it to save women.

Are we talking about two things?
One, is dominance necessary?
Second, what is the effect of this narrative to society, especially in a conflict zone?
Firstly, so what we have from opposition, a small number of materials, we won't dispute it because we also have a small number of materials.

One, they told us dominance is necessary because one, it incentivizes ICC to regard sexual rape as a crime, so we should have a discussion that we should also care about it.
Because people have further discussion and people have thought deeply about this woman's non-violence or vulnerability, we should include rape as a crime and so forth.
Firstly, we think that is wrong, still we don't understand this idea or this non-violence or vulnerability leads to rape being regarded as a crime itself.

I think the western male already knows, knew in the 1980s and 1990s that women tend to be non-violent and so are the liberal opposition, women tend to be vulnerable compared to men.
The reason I think, the more plausible reason is, they have been dismissing this idea of rape is because they have been quite male-centric, they don't care about this female gender in these developing countries right now.
I think they have been criticized a lot by the NGOs, the female movement, that even though you are saying that as an NGO or ICC that you are going to protect vulnerable, you are not really protecting those female who have been brutalized in the underground.

Second, they told us from the liberal opposition, the dominance is necessary because you can get more aid, so PKO can get more support.
Firstly, we already have told you that if the Red Cross needs a picture to incentivize people to donate, this you can have children.
What we heard from DLO is that no, having one child is less better than having children and wives. I don't get it.
I think having three children for having brutalized booking is tantamount to showing children mothers.
People don't make value judgment or difference on that point.
If there are lots of people who are children who have been brutalized for being so horrible, they are going to donate anyway on our side of the house.

So, let's talk about the impact for the entire population by sending cash.
We have told you from the Prime Minister that this actually leads to a post-conflict settlement.
And they told us, one, they're going to find an excuse for the leader of the opposition.
What our policy does is to make people get away and forget about these excuses to begin with.

We think it is necessary for the Western society to treat male and female quite equally and show the stance that regardless of the vulnerability, we should treat them equally.
They are an important source for the countries like Iraq and Afghanistan.
And we think we want those developing countries to follow the values that Western countries have.
That's why we think those people are more likely to think that sexism is something unfortunate, sexism is something wrong to have in our society as Western countries have done.

They told us, next was, no, this conservative does exist anyway, and that's quite minus, right?
We want to fix this conservative, moderate conservative, but as a Western country to show the stance that we want to treat those people equally, and you should treat those female and male equally, and forget about this conservative value we have in our society, and that we think more likely the developing countries follow our stance, yes.


So, here it looks like you regret the broadcasting which only changes stance and regretting of the theoretical dominant situation.
What do you support in comparison of how broadcasting has happened?
What sort of values should exist?

We want a society where broadcasts kind of become diverse, I guess.
We just want to oppose domination itself.
So, what we're saying is, maybe if the ones, for example, CNN broadcasts the way women have been brutalized, we're not opposing itself, right?
What we're saying is, the BBC, CNN, Russia Today, whatever, all sorts of media broadcasts that keep repeating the word, it is always the women and children who have been brutalized.
We want to avoid dominance itself, right?
We're not opposing to the idea of women being brutalized and broadcasting itself, right?
We think that may happen, but fine, because that's what this debate requires us to do, right?
We want to oppose dominant narrative itself.

## Opposition 3rd
Mr. Speaker, in order for government side's argument to stand in this debate, they must have proven two things, which actually are badly explained coming from government side.
First thing is, by removing his dominant narrative, women are going to fight against men, and successfully fight against men, and also men are going to hesitate to attack men to some extent, even though it's 100% men will change to some extent.

Second thing is, without this narrative, women are going to suddenly make contribution in conflicts, and they are admitted as legitimate actors and not recognized as secondary human beings or so forth.
But they never substantiated how women are going to contribute to a conflict society, in a conflict situation, and going to be rewarded, for example, after a conflict, even though they told us in so many situations there's already sexist context in their side of the house.

And also, I want to clarify what kind of narrative we are supporting from our side of the house.
Because government suddenly rejected the narrative they identified from Prime Minister.

From Prime Minister, they told us, we are talking about shocking image in broadcasting of Western media, and those kind of shocking image creating dominant narrative that women are vulnerable, women are non-violent, that is a setup coming from Prime Minister.
Then suddenly government said, it's about theoretical dominant narrative, women are vulnerable, and 100% they go against rape, or those kind of attack, and it's 100% safe for men to rape women, those kind of theoretical narrative from their side of the house.
I don't think it's realistic, right?

Because even though we have dominant narrative that women are vulnerable, it's not 100% safe.
Maybe sometimes women are going against rape, going to fight against men, and sometimes not.
That's exactly what we are talking about.
So we are arguing how shocking image, or those kind of messed up by MG, or those kind of Western media, is actually beneficial, harmful, for protection of people in need.
Also, those kind of depiction is better for post-conflict sexism that Prime Minister has set up recently.

With that being said, I'll talk about two things in my speech.
First of all, during conflict, this side can better protect people in need, including men.

And also, we talked about special care for women.
But firstly, it's inconsistent with Prime Minister, because they are saying that rape, women, or those kind of depiction is very effective for Western media to attract audiences in their side of the house.
And also, pragmatically, it's very difficult to differentiate children and women, because in a conflict situation, in many cases, children and women are running to each other, to other safe spaces, and they are detected to each other in their side of the house.

That's why we think, pragmatically, it's very difficult to differentiate women and children as the bounder walker in their side of the house.
That's why, Mr. Speaker, we think our side is even better for men, because total pile of AIDS will increase in our side of the house.
And also, it is going to increase more accessibility for men, because PKO and people in PKO don't say, because you are men, you shouldn't receive this kind of medicine and this kind of support.

Also, we told you that big narrative is not so important for militants there.
Anyway, terrorists kill many people.
Anyway, those kinds of conservative people will kill and rape women.
But we told you, at least militants in legitimate entity are carrying it because they recognize a crime against humanity or those things.

That's why, in a comparative manner, we think that we are trying to, we are successfully helping more women in the outside world.
So, above all, their case might be standing in law school level.
But after we explain the context and characterization of the situation of people there, their argument wasn't realistic.
In most chaotic situations in the world, they don't care about those kinds of human rights.

From their side of the house, they said the reason why women are less treated in the post-conflict society was that male victims weren't safe while women are already safe.
And also that men feel they deserve reward or higher rights because they contribute to the ends of conflicts than women.
But, Mr. Speaker, regardless of the existence of this narrative, usually women have relatively more difficulty in contributing to the conflict or humanitarian crisis in this military purpose of physical contribution.

Because just because we remove those kinds of narratives, women are not suddenly going to participate in militaries or not trying to solve the humanitarian crisis in their side of the house.
So, which is better?
In their side, women are recognized as non-vulnerable and maybe sometimes violent, but they are not contributing to the conflict.
But in our side of the house, women have justification not to contribute to the conflict because they are vulnerable, they are physically comparatively weaker.
That's why they systematically couldn't contribute to the conflict.
That's why, even though they didn't contribute to the conflict, they are not recognized as irresponsible actors or those kind of things.
We told you how, right?

And also they are saying that in post-conflict society, women are targeted on the street and women at all, not to wear slutty clothes and those kind of things.
But we are not talking about those kinds of narratives.
Because we are talking about narrative in a conflict, right?
Maybe when it comes to real society, when it comes to safer places, there are different dominant narratives.
Women can do anything that they want.
We are talking about the narrative during the conflict situation.
For that reason, we think even though logically there is some understanding, we think even though post-conflict female right, we are staying outside the house.

## Opposition 4th
Two very simple questions. Is the dominance of the NIDILSC important and better assured?
Second, could the dominance be toxic? So, from there, we told you that the dominance of the NIDILSC is important in three variety of ways.

We firstly told you that it's better for the check and balance of the conflict zone, like we're able to establish the condition of rape in ICC.
In similar ways, we have numerous check and balance institutions going on in conflicts and situations of humanitarian crisis.
More pressure would lead to better institutions and check and balances of enforcement of those institutions.

Their response from government was that, you know, it's probably a different reason, probably was male-centric and was able to tackle that.
But they don't deny the fact that it was ultimately pressure that changed the situation.
What we told you is not that the reason doesn't matter. We told you that this vulnerability leads to more social pressure to check on the situation of women, and that social pressure collectively moves the government, collectively moves international organisations, collectively moves the United Nations to change conditions.

Second, we told you that it creates at least more aid. The pie is an exclusive thing, they get less.
Their response is that, you know, children are enough to create sympathy, but also people don't care about the oily aid, which my partner dealt with.
It's that the combination of children and women are the relative context, and that's why people are more able to sympathise with it.

But they don't engage with the nuance of the fact that some people might sympathise with children, but you have to recognise some people also increasingly sympathise with women more than children.
We are able to grab the minds of these people, which means that we are able to have more people driven to say that they want to send goods to these areas, they want to donate to these areas, and things like that.
So children might be sufficient, but they miss out on important individuals and contexts, like feminists or other individuals.

But also, see, it also lacks a nuance when we tell you that, you know, it's women that are particularly vulnerable because they are associated with sexual bullying, like rape, or gruesome attacks and things like that.
That sort of narrative is impossible when you only use children.
That is the most tragic and constant, detrimental situation that people don't even imagine, and that's why they want to help the situation.

They are moved viscerally that they have to help the situation. That is why women are particularly an important tool in showing the vulnerability of the situation, and it simply does us very negative in the attempt to say that it's no different.
Since they have a unique power, we get more power to provide aid and whether it's a better issue.
But we also told you that it's important that we have a better understanding of the conflict situation.

It's not that you need more firepower to protect against guerrillas, you need charity provisions, you need more protection about privacy, or better allocation of refugees to avoid rape between men attacking women and things like that.
And their response was that, you know, it's the NGOs that have to understand it.
But that's not the point. What we're telling you is that these NGOs also might be inexperienced or unaware of what is actually necessary.

What is it compared to the substantiated harms that we provided?
It's all about sexism, terrorism and things like that.
We told you there are a variety of other reasons for that to happen. We don't understand why this scenario was particularly necessary and why it's the strongest thing.

We also told you that particularly men feel responsibility and pride in the fact that they help women.
It would be a shame if they say that women deserve to contribute less and it's okay that we feel wrong for women that they didn't contribute.
They feel pride in the fact that they're helping women and going on with it. There was no response to that characterisation.

But further, there was no response to us saying that this debate, as I said, is about westernised.
We are not sure why this conflict on the humanitarian crisis area is looking at BBC or the exposure of Naira in the first place.
That's why when they talk about the mindset being changed by the Naira about terrorists or people on the ground, they didn't actually prove the connection.

## Government 4th
There was a news report showing the video of Syrian child who was drowning in the sea.
That kind of video, it was sufficient and efficient enough in order to move public opinion toward acting against the refugee crisis.
We believe those pictures or the video of a child or one woman or one man brutally being killed or brutally being injured is something efficient and sufficient enough in order to move public opinion.

Their word of proof is that Y-dominant narrative of women being exposed in the westernized media is something important in order to gain money, in order to gain donation, in order to gain humanitarian aid, which was never proven from the side negative.
We already told you from the deputy prime minister and government that that is not enough to win today's debate because this kind of dominant narrative means that BBC, CNN, those kinds of New York Times, they are always saying that women are being the target, women are being the victims, women being vulnerable, women being non-violent.
That is the dominant narrative that they have to prove in today's debate.

So we gave you two problems from the affirmative.
Firstly, how the dominant narrative affects female's lives.
We told you that they are used as a tool of the terrorist groups.

Opposition told us that terrorists do not depend on such method or tactics.
For example, they told us about the suicidal bombing or those kinds of things.
Mr. Speaker, that is only limited to ISIS or those kinds of things.
But we already have those kinds of Boko Haram using those kinds of female's lives in order to send those kinds of political messages.
We believe those kinds of examples do not make them win this point.

But secondly, we told you that the dominant narrative creates sexism in post-conflict society.
Deputy leader of the opposition said that men do not do such kind of sexist actions because they have a duty to protect women.
But Mr. Speaker from the leader of the opposition, he said that they are going to create another excuse in order to continue doing those kinds of sexist actions.
Well, there is a contradiction in those kinds of characterizations of the conservative males.

Therefore, Mr. Speaker, they could not prove why these post-conflict sexism do not happen in post-conflict society.
They could not prove those kinds of points.
Therefore, given that, opposition told us that the post-conflict sexism is much less important than immediate protection of female's lives.

Well, Mr. Speaker, they are underestimating the power of sexism.
It can lead to honor killing.
It can lead to stoning.
It can lead to the threat of female life, Mr. Speaker.
We live in that kind of situation.
That is not a mere dichotomy between lives and mere sexism.

This kind of sexism cannot easily lead to the death or the threatening of female's life rights.
We believe that is horrible, right?
Therefore, Mr. Speaker, what we told you in today's debate is that, one, those kinds of post-conflict sexism is something horrible, something that can threaten female's lives.
And second of all, we told you that the dominant narrative is affecting the current female's rights, the current female's lives.
They are making terrorists attack females.
They are making terrorists get the records or to get the opportunity to kill females, to rape females in the current status quo.
We believe, what they told us, is that not mutually exclusive.
Their benefit to gain money and gain more prize and allocate to females is not mutually exclusive.
That can be done by children.
That can be done by male victims who are brutally killed, brutally injured.
We believe they cannot fulfill their burden of proof.
We win this debate.
"""

# reasoningモデル
response = client.responses.create(
    model="o3",
    reasoning={"effort": "high"},
    input=[
        {
            "role": "user", 
            "content": prompt
        }
    ]
)

# 応答の表示
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-4]
log_filename = f"logs/log_{current_time.replace(':', '-').replace(' ', '_').replace('.','-')}.txt"
with open(log_filename, "a", encoding="utf-8") as log_file:
    log_file.write(f"Timestamp: {current_time}\n")
    log_file.write(f"Response detail: {response}\n")
    log_file.write(f"Reasoning Summary:\n{response.output_text}\n")
    log_file.write(f"Input Prompt:\n{prompt}\n")

print(response.output_text)

Score: 3 / 4

Explanation of the score – how well the benches actually engaged with one another’s material.

1. Quantity of clash  
   • All of the big questions opened by Gov-1 (terrorist targeting, post-conflict sexism, unconscious sexism and unequal aid allocation) are at least mentioned and answered by Opp-1.  
   • Every subsequent speaker continues to reference the other side’s previous speech rather than simply adding brand-new extensions. The argument pairs (“aid needs a ‘vulnerable-women’ hook” vs. “any shocking image works”; “narrative invites extra terrorist attacks” vs. “terrorists already have multiple stronger incentives”) run through the entire debate.  
   • Both whips devote most of their time to line-by-line rebuttal and comparative weighing, not to inserting new material. This shows genuine interactive clash rather than parallel monologues.

2. Directness and specificity of rebuttal  
   • Rebuttals usually quote or paraphrase the precise mechanism they are answering

# message model

In [ ]:
# from dotenv import load_dotenv
# import os
# from openai import OpenAI
# from datetime import datetime

# # .env ファイルの読み込み
# load_dotenv()

# # APIキーの取得
# api_key = os.getenv("OPENAI_API_KEY")
# if api_key is None:
#     raise ValueError("環境変数 'OPENAI_API_KEY' が設定されていません。")

# # OpenAIクライアントの初期化
# client = OpenAI(api_key=api_key)

# # text = """
# # # Instruction
# # Please evaluate the following parliamentary debate round based on the degree of engagement of arguments. The motion for this round was "This House regrets the dominant narrative of women as non-violent and vulnerable in conflicts and humanitarian crises." Return the score from 1 to 4.
# # # Debate Transcripts
# # ## Government 1st
# # Intention of the Western media, like revealing this dominant narrative regarding the vulnerable women, they don't have the malicious intention, but they rather have good intention in order to save women, save children.
# # But the outcome is different, Mr. Speaker.
# # Because they target, these women are vulnerable, they target these children are so vulnerable that they are weak, they are non-violent.
# # That leads to the bad outcome that ultimately these women and children are being targeted by the terrorists, are being targeted by those post-conflict sexism.
# # We believe that is something horrible, as the Western media has done.
# # We believe, we beg to propose.

# # I'm going to talk about mainly three things.
# # Firstly, how this dominant narrative badly influences post-conflict zone sexism.
# # Secondly, how this dominant narrative is badly affecting these women and children regarding these kinds of terrorist attacks.
# # Thirdly, and lastly, how this dominant narrative leads to unconscious sexism.

# # First thing first, about the dominant narratives.
# # What we mean by dominant narrative is those, mainly those Western medias, like picturizing these women and children who are being raped, who are being brutally murdered in the conflicts or the humanitarian crisis zones.
# # They picturize those women and children being forced by the power of male, and being forced by the power that they are being raped and they are being weak toward those kinds of attacks.
# # They have the massive incentive to use women and children as a vulnerable and non-violent actor.
# # It's because it's pretty shocking that they are using the picture of women being raped, picture of children being stabbed by the knives or those kinds of things.
# # Because those kinds of shocking pictures and shocking words are going to raise the viewing rate or those kinds of magazines or those kinds of things.
# # Therefore, those Western media has a massive incentive to make those kinds of narratives very dominant.

# # As a consequence, by this kind of dominant narrative done by Western medias, there will be much more attention toward female victims than the male victims, Mr. Speaker.
# # For example, in Boko Haram, for example, in Syria, for example, in ISIS areas.
# # As a consequence, what happens is that there will be a massive amount of discussion inside of U.S. councils or WTOs or those kinds of more policies sending peacekeeping operations or humanitarian aid toward those kinds of women and children who are dying on the ground inside those kinds of conflict zones.
# # What happens is that peacekeeping operations and humanitarian aid enter conflict zones and prioritize women and children than the men.

# # What happens is that more male victims are going to increase rather than those kinds of female victims, Mr. Speaker.
# # This leads to the post-conflict sexism, Mr. Speaker.
# # What do I mean by this?
# # It means that those kinds of male victims who are brutally killed or male victims who are brutally injured by those kinds of war operations, they feel that they need a reward in a post-conflict society.
# # They need some kind of class.
# # They need some kind of money.
# # They need some kind of reward that they have done in conflict zones.
# # Therefore, they cannot give away their money or their tax revenues toward women because they think that they were already saved by the Western society.
# # They were already prioritized than our lives in the conflict zones.

# # We believe in that kind of scenario, that leads to the post-conflict sexism, Mr. Speaker, that those men are getting a reward and women are getting nothing in return because they have already got those kinds of reward or the treatment during those kinds of conflict zones.
# # We believe, rather, that saving those kinds of women and men equally leads to the betterment of these kinds of situations because those kinds of male victims cannot make the excuse that they were de-prioritized than female victims and therefore they cannot use those kinds of rhetoric.
# # We believe this kind of sexism is going to better off on our side of the house.

# # Secondly, Mr. Speaker, those kinds of dominant narratives lead to more targeting women and children by the terrorist attacks.
# # Why is that?
# # It's because those kinds of terrorists, for example, ISIS, knows that Western media uses female and children as a news title or those kinds of eccentric news reports.
# # Therefore, they have a massive incentive to spread their political messages throughout those kinds of news outlets, which means that by targeting women and children, they will have a massive potential or possibility to make their political messages throughout those kinds of Western media.
# # Because they know that Western media have an incentive to use and target those female victims' reports or the children victims' reports.

# # Therefore, what those kinds of Western media is doing is the opposite of their aim, Mr. Speaker.
# # They are trying to target those women and children and prioritize their lives, but what they are doing is the opposite.
# # They are rather giving those kinds of opportunities for those terrorist groups to attack these women and children.
# # We believe that is morally horrible.

# # Thirdly and lastly, why this dominant narrative leads to unconscious sexism.
# # Dominant narrative regarding women who are pictured as weak and non-violent actors is like 100,000 women who were brutally killed and raped.
# # They were forced by men's power.
# # They were overly done by Western media and that leads to unconscious sexism.

# # So why is this bit of more sexism more important than the increased amount of aid or aid going to people who actually need it?

# # Because, Mr. Speaker, first of all, we want to question why do you prioritize women's lives over the male, right?
# # We could see why that is justified in the first place, but second of all, we do not want, by equalizing those kinds of aids and treatments, we believe that the post-conflict society is going to be better off, right, better off, Mr. Speaker, because sexism is something really rooted in the society and that is going to give less rights to women, Mr. Speaker.
# # We believe that is important as well.

# # So by picturizing women as weak and non-violent actors, that is going to make people unconsciously sexist toward women.
# # They are going to say that because you are weak, you cannot wear slutty clothes, you are going to be raped in the town, you cannot wear those kinds of slutty clothes because you are weak, right?
# # Those kinds of narratives are going to be dominant and that is going to lessen women's rights and freedom.

# # ## Opposition 1st
# # Okay, two issues from leader position.
# # Firstly, I will talk about the realistic characterization of women in conflict.
# # We do recognize that some women are strong, who are capable enough to fight back against men.
# # But unfortunate reality is that in general, women are likely to be comparatively more vulnerable than men,
# # and comparatively more likely to target the atrocious consequences.
# # Thus, they need more support and protection.

# # Secondly, I will talk about why, due to the dominant narrative,
# # they uniquely make it possible for us and surroundings or NGOs or donors
# # to provide the necessary support and a decent amount of support
# # so that their women are going to be protected.

# # Before that, I will present the responses to the Prime Minister.
# # Firstly, they said that this dominant narrative incentivized terrorists to exclusively target women in the conflict areas.
# # Firstly, we do concede that the status quo is not perfect,
# # but we think that due to this narrative, the situation is currently going to be changed.
# # For example, the International Criminal Court actually introduced the rape or sexual assault as crimes against humanity,
# # and the international recognition that sexual assault requires special recognition.
# # And also, there is a certain international criminal bashing towards the ignorance of the government
# # who has an incompatibility to deal with sexual assault.

# # But furthermore, when it comes to the media's emphasis on gaining attention,
# # we see that terrorist groups are also already relying on the other kind of things.
# # For example, they are relying on the suicidal attack, or they are relying on the indiscriminate killing,
# # and this is even more attracting when it comes to getting attention through the media.
# # So I don't think this is a core motive for terrorist groups to exclusively target women on the ground.

# # They said that male protection is going to be on the mind, compared to the female.
# # But as I explained earlier, we think that given that women are likely to need special protection,
# # we think it's proportionate and we think it's good.

# # Lastly, they said that this dominant narrative is leading to the post-conflict system.
# # But we think that if those kinds of leaders or men in society have the intention to discriminate women,
# # we think that they are going to anyway find any excuse to discriminate women or let's reward women on the ground.
# # Maybe, for example, they may say that due to their pregnancy, women cannot, unfortunately,
# # work exactly the same as men or so forth.
# # Anyway, sexists are going to find that kind of excuse and discriminate women.

# # But furthermore, even in that case, we have to be comparative.
# # While it's also a kind of social position of the post-conflict society,
# # it's kind of more important than the immediate help of the women in the conflict zone.
# # We think it's more important.

# # So I'll just talk about the realistic characterization of the women in the conflict.
# # So in general, women are fortunately likely to be comparatively more valuable than men for several reasons.
# # Firstly, physically, women are kind of likely to be weaker or less physically capable to fight back exactly the same level as men.
# # It's just that women are likely to be the target of the conflict.
# # But furthermore, we believe that women are exclusively likely to be the target of sexual assault on the ground.
# # We do recognize that men also can be the victim of sexual assault.
# # But we think the rate is disproportionately higher on the male and female.
# # If we look at the reality, for example, we come to the abduction by the terrorist group like Kohara.

# # Most of the victims, the abductors, are teenage girls in the school, and they are abducted by the terrorist group.
# # So we think in the realistic context, those who are an individual, women are more likely to be the target.
# # We think they need special protection.
# # But furthermore, maybe the need for sanitary protection due to the biological feature.
# # Maybe, for example, there is a pregnant mother who needs much more protection, who is more vulnerable, etc.
# # So we think that realistically, women are likely to be relatively more vulnerable, thus they need special protection and support.

# # So given the nature, let's look at the second issue about why the dominant narrative is necessary in order to protect these women on the ground.
# # So we think, firstly, that the dominant narrative allows the point of peacekeeping operation, intervening military or any other individual, etc.
# # to provide more support to the women on the ground.
# # Maybe, for example, we come to this shelter, which has a lot of women.
# # Maybe we can deploy more soldiers so that we are able to protect them more.
# # Or, for example, we are giving extra support, medical support or sanitary support to pregnant women, etc.
# # And also extra support is going to be available on our side of the house.
# # And also, as I already told you, ICC's recognition of rape or sexual assault as a crime against humanity
# # are also introduced due to this dominant narrative that recognizes that women are relatively more vulnerable on the ground.
# # And we think it's beneficial.

# # Secondly, we think that this dominant narrative allows the...
# # What is the connection between ICC recognizing sexual rape as assault and making people believe that women are non-violent?
# # This is because, due to the dominant narrative, people in the liberal democracy can't stand women are more likely to be victimized than any other assault in Korea, etc. No thank you.
# # So we think, yeah, that's really important to say that.

# # But secondly, we think that due to this dominant narrative, it kind of enables the individual donors in the liberal democracy or the advanced nation
# # to think about the women, the situation of the women on the ground, and think about what kind of support they are going to need, etc.
# # And we think it's critically important, right?
# # Because the majority of the average citizen in the liberal democracy haven't experienced the situation of the conflict that much,
# # and it's kind of hard for individuals to imagine what kind of support they are going to need, etc.

# # And we think that the dominant narrative allows those average citizens or individuals who can be the potential donors to the NGO
# # or the supporter towards the women on the ground, right?

# # And they are going to think that, you know, that women may need, for example, certain special, like, sanitary protection,
# # or maybe can't, like, want to need extra protection, extra, like, deployment to the military due to the high tendency of sexual assault.
# # And we think that it's also the recognition of what, like, understanding of the exact necessity of support is very important, right?
# # Because in order to, like, maximize the utility or, like, benefit from the support and the donation,
# # we think that this, like, dominant narrative is critically important.

# # So overall, we think that, you know, given that it's, like, unfortunately that women are more likely to be vulnerable and targeted at a special time,
# # we think that it's, like, beneficial and reasonable to use the dominant narrative to gather support and help them.

# # ## Government 2nd
# # The opposite side claims that without this dominant narrative that women are non-violent in conflict zones.
# # We are not able to attract the media's attention, the western's attention, and the NGOs in the west are not going to be able to protect the people who are suffering in conflict zones.
# # But we reject this analysis.
# # We think that there's a lot of people who are suffering in those conflict zones who are not women.

# # For example, maybe men's child therapy, their hands, because of this really aggressive guerrilla groups.
# # We think those kind of images can actually shock people and feel that there's a necessity to actually reach out and protect them.
# # We don't think that that dominant narrative of women suffering is something that's necessary.
# # There's a lot of children who are suffering, men suffering, and also women.

# # We think that we can't have women suffering, but we just can't have all kinds of people just show that all kinds of people are suffering.
# # We think that in itself is something that is not.
# # We think that this kind of image that we are creating, that women are weak, women are vulnerable, actually makes women unable to actually fight back.
# # Not only during the conflict zone, not only during the conflict zone, but also after the conflict zone.

# # And we think we are making the women in a weaker position, so that is why we are very happy to propose this motion.
# # So I'm going to maybe talk about two things.
# # First, I'm going to talk about why it is still going to go to those kind of conflict zones.
# # And secondly, I'm going to talk about which is better for the female.

# # But before moving on to my points, I have a refutation to their side of the house.
# # So the leader of the option refuted to my partner that men will always make an excuse to actually exploit women.
# # But we think that may be true, but if there's less excuses, that means that there are less reasons that they can justify their actions.

# # So we think that if there are less excuses, for example, women can fight back and say that that is not right because I also contributed to the society.
# # I also fought back.
# # I also contributed during the conflict zone, so I should have the right to decide on my political will.
# # I should have the right to decide to go outside because I can actually protect myself.

# # So first, I want to talk about why aid is still going to go in those kind of conflicts.
# # So even if there's not a dominant narrative, that women are not violent and vulnerable.
# # So we were talking about like, so they talked about we have to show images that women are getting raped, that women are getting, like for example, there's need to dominate narratives of women being harmed and raped or those kind of things.

# # But we think that in places where in conflict zone, we think that not only women are suffering of getting raped.
# # For example, in places like in Congo, there are many children and many also men getting killed brutally.
# # Maybe they're cut their limbs because they think that cutting limbs would actually make their opponents weaker.
# # And we think that those kind of brutal images can be shown even in our side of the house.

# # And we think that if media can find other images that are very shocking, they have an incentive to do it.
# # So we think that even if women are not being raped and they don't show a dominant narrative of women being raped, we think that it can also attract the media's attention if some people are actually suffering.
# # And we think that because there's an overt dominant narrative that women are getting, like for example, suffering, we think that the amount of resources used to save the men or other people are not going to be used.
# # And we think at the end of the day, we think that the amount of people that can be saved are going to be the same.

# # We think on our side, maybe fewer women are saved.
# # We can see to that.
# # But at least maybe more men are going to be saved because of this less dominant narrative.
# # And we think that in itself is not really a bad thing.
# # And we don't understand why they want to prioritize women over men.
# # We think all lives are equal.
# # And if men are maybe more protected, we think that in itself is not really a bad thing.
# # And we think there are a lot of men who are suffering.
# # And we think at the end of the day, the number of people who are going to be saved is the same.

# # Secondly, I want to talk about which is better for the female at the end of the day.
# # So firstly, about during the conflicts.
# # So we think that because we actually internalize this, because NGOs are in the West, actually prioritize protecting women.
# # We think women have less capacity to actually protect themselves.

# #What do I mean by this?
# Because there is no necessity for women to actually maybe arm themselves because the West is there to protect them.
# And because the West thinks that you have to be protected.
# We think that is why women themselves do not arm themselves.
# They said that women are vulnerable.
# So that is why women are not able to protect themselves.

# But we think this narrative in itself actually makes women vulnerable.
# We think women in itself can actually protect themselves.
# So if you have a gun, even if your man is much more masculine and has much more physical strength, you can kill that another person.
# So we think that physical strength in itself is not a matter in this debate.

# So we think that those women can arm themselves.
# And when we feel that they are capable to arm themselves, they won't be able to do it.
# Because this dominant narrative is so strong, they are not able to arm themselves.
# They think that they are vulnerable.
# They lose hope and protect themselves.
# And that actually leads the women to be targets of, for example, a lot of terrorist groups.
# Or they are getting targets of getting raped.
# We think this narrative in itself makes women weak.

#  Are you saying if this narrative doesn't exist, can you make women suddenly take guns and fight against men and protect themselves?

# I'm talking about being comparative.
# I don't think this is an absolute solution.
# But we have to be comparative.
# That at least women will feel that, oh, I have to protect myself on my own.
# And men will fear that, oh, if I actually attack this woman, maybe I might get killed.
# We think that people's thinking is going to change.
# And we think that women are going to be less targets.
# And we also think that there are going to be fewer targets of, for example, Boko Haram or ISIS.
# And we think they never actually refused to do that.

# And secondly, after conflicts.
# I'm going to talk about after conflicts.
# We think that in the status quo, women are seen as vulnerable individuals.
# And they are seen as people who did not contribute during the conflict zone.
# Maybe they did house chores.
# Or maybe they supported it.
# But they are seen as secondary individuals, second-class citizens.
# Because there was less contribution.
# And because we are seeing that these people are vulnerable and they did not contribute, we are making society can create a norm.
# The woman is less contributing, so let's get them fewer rights.

# And we are also sending a message.
# Being supportive, for example, of the people who are fighting.
# Or doing house chores is also not a good contribution.
# It's not something that is a significant contribution.
# And that makes women much more vulnerable after conflicts.
# And we think that in itself is bad for the women.

# So that's why we are very happy to propose this motion.

# ## Opposition 2nd
# Okay.I'll bring you two new substantives.
# First, why the use of a vivid and vulnerable activity to increase attention and material support to areas.
# Why the piling of the aid and support going to the areas is not the same in two paradigms but it's increased on our side of the house because we have more vulnerable, clear images to the public where people feel more need and sympathy to support these areas.
# Second, I'll talk about why this dominance of a narrative is important.
# Why people should be aware of the image of what women actually need.
# Why that pre-imposed exposure to the idea is necessary for people to, in that chaotic, immediate need to help people, they are able to understand what donation is necessary, what they should act right now.
# For inexperienced NGOs who go into the area, knowing about the situation is so important.

# Before I go on, a couple of independent rebuttals.
# So, first, they talk about how, you know, it leads to some sort of sexism because, you know, they're vulnerable or something like that from the Prime Minister.
# They told us what sort of sexism and action it actually leads to and I think the logic is very blurred.
# And also, we have to recognize that, you know, in these areas, or it's universally, helping women and children first as a man is a pride of a man.

# It's seen as a responsibility of a man.
# Also, in any form of conservative religion, it's seen that it's the responsibility of the man to protect women and children first.
# That also adds to the fact that they won't see that they are deprioritized or they are being abandoned.
# They see it as a pride that women are actually given more support.
# They will actually facilitate to let women go on with it.
# So, it's wrong for them to say that men are so greedy and go on with it.
# We think that those men do exist, but they're a minor part of society.
# I think that's a bizarre characterization.

# But also, even if we concede that sexism might happen to a certain scenario, we think that's far less important than people being saved by the fact that they're given sanitary goods.
# They're saved by the fact that they're given food and clothing or protection from, I think, mosquitoes and things like that.
# So, it's only possible because they have a substantial amount of aid provided to these people.
# And we think that, talking about the larger individuals in that vulnerable situation, this bit of blurred sexism is far less important than that.

# Then, they also talk about, from FPM and BPM, that they are seen as vulnerable, so terrorists are likely to more attack them.
# Or men, I like to say that they're weak, so you should obey us or use it as a rhetoric for sexism and things like that.
# Firstly, I think Errol talked about how terrorists don't really care about it, which was not responded.
# But further, there are plenty of other reasons for terrorist organizations or these men to be sexist on women.

# Firstly, there are reasons to say that, you know, for terrorists, they are intuitively weak in the physical sense.
# They may have greed for sexual desire, so there are other reasons to attack women first, unfortunately, as men, male soldiers.
# Or they simply think that they have the extremist views of, for example, Islamic beliefs that say that they should subordinate women, so they should attack first instead of men.
# Or they might have this strategic interest attacking women groups, this historical know-how as tribes and things like that.
# And given the fact that terrorists often don't really care about it, it's that this additional one reason of a narrative doesn't really change their calculus because they already have tens of other reasons to do that.

# If the tens of other reasons don't change their calculus, I don't understand why this bit of a narrative that these terrorist people haven't been exposed to, maybe, has any influence to it.
# And even if not, because even if you add it, because there's not much of a difference, the substantial difference is really unclear.
# Even if you make male, like, trying to make it obedient, you know, this sexist belief or preferences of men to go on with it already exists.
# This bit of narrative that exists in the context also has no clear...
# Sorry.
# It amplifies that narrative, but it's not clear to what extent that's important.

# No, thank you.
# But also importantly, this narrative, as they set up, is a broadcasting of Western media.
# It's not about how the narrative forms in the conflict zone, but rather about people around the area.
# Not in the area, but around the world.
# And we also think that also proves the fact that the influence they want to suggest is not that huge.
# Sure.


# So, why have you told us that these conservative sexism are quite minor, limited to not minor people?
# Now, you're saying that there are lots of conservative sexism does exist, so women won't be able to get out of it.
# Which is it?
# There are a lot more conservatives or less conservatives?

# I'm saying that if they do exist, it's that it's not going to be rid of a big issue.
# So, no, we're not saying that they're a majority.
# I don't really get it.
# Connection Between ICC and Dominant Narrative
# Then there's a bit about ICC and things like that.
# We understand that there's not clear connection between ICC's recognition of rape and vulnerability of women.
# The thing is that it was necessary for the discussion and investigation of female issues even at core, that we needed social pressure to have women being protected.
# Because in the past, the PKOs, the reporting, didn't even have a section for gender or women.
# We didn't really investigate the rape.
# It was all just about protecting against guerrillas and things like that.

# Why is the use of vivid images so important?
# Importantly, the support to these areas are vital to the survival of people on the ground.
# It might be aid, it might be interventions, it might be sanctions, but all of this is contingent on the fact that you have a necessary critical mass of people who care about the area, who are angered by the situation and want to support these areas.
# The more support and attention you have, you have more people saved, or quickly saved because of interventions that were done a bit more quickly.

# Obviously, the sympathy that is created by the image of the media is enhanced when you use, unfortunately, more vulnerable looking actors.
# The use of the image of a woman is far more vulnerable looking than a man.
# Also, religious narratives and social narratives about how men should be obliged to protect women also enhance the fact that when women are put in vulnerable positions compared to men, it provokes some emotional feelings that they should protect them compared to men.
# They do have the tool of children in their side of the house too, but the mixed use of women would also lead to an accelerated power to have more attention in these areas, and an increased amount of power means that we are able to save these people on the ground more, and that means that more aid is coming around the side of the house.

# So they say that men are going to receive less aid, but if the pie is increased, they don't receive less.
# But also, even if we can see that men don't receive sufficient aid, it's that men have more energy and have vitality compared to women.
# So being prioritized in the situation of aid is far less important than women being provided of the actual needs on the ground.
# Shortly on why dominant narratives.

# Unfortunately, people, when they want to actually help a certain area, they just delay the things that they actually think they need.
# But the clothing and the food that they actually provide might not be a necessary thing, it might be a burden to the actual area.
# To prevent that, it's important that people expose the image of what actually women need.
# The image that women are suffering in this particular way through the media, providing the understanding that the issues are real, the issues are sanitary conditions, you need napkins instead of actual random foods or clothing and things like that.

# That's only possible when the dominant narrative is also public and people are pre-emptively aware of the particular situation.
# Inexperienced NGOs also will be able to be aware of the situation and not provide random unnecessary things but actual needy things for women.
# That's only possible if we can have a narrative and not an alternative systematic approach.
# Thank you.

# ## Proposition 3rd
# This debate is not about whether we are allowed to put this picture as, for example, MSF, like for example Red Cross, showing a picture that women are being brutalized in African countries.
# This debate is about whether that should be dominant, whether that narrative should be dominant, that everyone believes that it is always a female who are being brutalized.

# It is always a female who are never going to be violent, who are never going to be non-vulnerable.
# Women are always weak, they cannot do anything by themselves, and that narrative should be dominated within a society and make every person in a society, including the white men in the United States or the female in Africa, to believe that women should be non-violent.
# We do not think so because that obviously does not reflect the reality, and we think that incentivizes multiple kinds of actors to believe also that women are non-violent and vulnerable, they won't fight back.

# We can utilize and we can exploit the weakness which has been created by the Western media to harm women.
# We think saying that women are non-violent and vulnerable does not protect women, they are going to be more targeted, they are going to be exploited of the weakness which has been created by media, we think that is wrong, we should go against it to save women.

# Are we talking about two things?
# One, is dominance necessary?
# Second, what is the effect of this narrative to society, especially in a conflict zone?
# Firstly, so what we have from opposition, a small number of materials, we won't dispute it because we also have a small number of materials.

# One, they told us dominance is necessary because one, it incentivizes ICC to regard sexual rape as a crime, so we should have a discussion that we should also care about it.
# Because people have further discussion and people have thought deeply about this woman's non-violence or vulnerability, we should include rape as a crime and so forth.
# Firstly, we think that is wrong, still we don't understand this idea or this non-violence or vulnerability leads to rape being regarded as a crime itself.

# I think the western male already knows, knew in the 1980s and 1990s that women tend to be non-violent and so are the liberal opposition, women tend to be vulnerable compared to men.
# The reason I think, the more plausible reason is, they have been dismissing this idea of rape is because they have been quite male-centric, they don't care about this female gender in these developing countries right now.
# I think they have been criticized a lot by the NGOs, the female movement, that even though you are saying that as an NGO or ICC that you are going to protect vulnerable, you are not really protecting those female who have been brutalized in the underground.

# Second, they told us from the liberal opposition, the dominance is necessary because you can get more aid, so PKO can get more support.
# Firstly, we already have told you that if the Red Cross needs a picture to incentivize people to donate, this you can have children.
# What we heard from DLO is that no, having one child is less better than having children and wives. I don't get it.
# I think having three children for having brutalized booking is tantamount to showing children mothers.
# People don't make value judgment or difference on that point.
# If there are lots of people who are children who have been brutalized for being so horrible, they are going to donate anyway on our side of the house.

# So, let's talk about the impact for the entire population by sending cash.
# We have told you from the Prime Minister that this actually leads to a post-conflict settlement.
# And they told us, one, they're going to find an excuse for the leader of the opposition.
# What our policy does is to make people get away and forget about these excuses to begin with.

# We think it is necessary for the Western society to treat male and female quite equally and show the stance that regardless of the vulnerability, we should treat them equally.
# They are an important source for the countries like Iraq and Afghanistan.
# And we think we want those developing countries to follow the values that Western countries have.
# That's why we think those people are more likely to think that sexism is something unfortunate, sexism is something wrong to have in our society as Western countries have done.

# They told us, next was, no, this conservative does exist anyway, and that's quite minus, right?
# We want to fix this conservative, moderate conservative, but as a Western country to show the stance that we want to treat those people equally, and you should treat those female and male equally, and forget about this conservative value we have in our society, and that we think more likely the developing countries follow our stance, yes.


# So, here it looks like you regret the broadcasting which only changes stance and regretting of the theoretical dominant situation.
# What do you support in comparison of how broadcasting has happened?
# What sort of values should exist?

# We want a society where broadcasts kind of become diverse, I guess.
# We just want to oppose domination itself.
# So, what we're saying is, maybe if the ones, for example, CNN broadcasts the way women have been brutalized, we're not opposing itself, right?
# What we're saying is, the BBC, CNN, Russia Today, whatever, all sorts of media broadcasts that keep repeating the word, it is always the women and children who have been brutalized.
# We want to avoid dominance itself, right?
# We're not opposing to the idea of women being brutalized and broadcasting itself, right?
# We think that may happen, but fine, because that's what this debate requires us to do, right?
# We want to oppose dominant narrative itself.

# ## Opposition 3rd
# Mr. Speaker, in order for government side's argument to stand in this debate, they must have proven two things, which actually are badly explained coming from government side.
# First thing is, by removing his dominant narrative, women are going to fight against men, and successfully fight against men, and also men are going to hesitate to attack men to some extent, even though it's 100% men will change to some extent.

# Second thing is, without this narrative, women are going to suddenly make contribution in conflicts, and they are admitted as legitimate actors and not recognized as secondary human beings or so forth.
# But they never substantiated how women are going to contribute to a conflict society, in a conflict situation, and going to be rewarded, for example, after a conflict, even though they told us in so many situations there's already sexist context in their side of the house.

# And also, I want to clarify what kind of narrative we are supporting from our side of the house.
# Because government suddenly rejected the narrative they identified from Prime Minister.

# From Prime Minister, they told us, we are talking about shocking image in broadcasting of Western media, and those kind of shocking image creating dominant narrative that women are vulnerable, women are non-violent, that is a setup coming from Prime Minister.
# Then suddenly government said, it's about theoretical dominant narrative, women are vulnerable, and 100% they go against rape, or those kind of attack, and it's 100% safe for men to rape women, those kind of theoretical narrative from their side of the house.
# I don't think it's realistic, right?

# Because even though we have dominant narrative that women are vulnerable, it's not 100% safe.
# Maybe sometimes women are going against rape, going to fight against men, and sometimes not.
# That's exactly what we are talking about.
# So we are arguing how shocking image, or those kind of messed up by MG, or those kind of Western media, is actually beneficial, harmful, for protection of people in need.
# Also, those kind of depiction is better for post-conflict sexism that Prime Minister has set up recently.

# With that being said, I'll talk about two things in my speech.
# First of all, during conflict, this side can better protect people in need, including men.

# And also, we talked about special care for women.
# But firstly, it's inconsistent with Prime Minister, because they are saying that rape, women, or those kind of depiction is very effective for Western media to attract audiences in their side of the house.
# And also, pragmatically, it's very difficult to differentiate children and women, because in a conflict situation, in many cases, children and women are running to each other, to other safe spaces, and they are detected to each other in their side of the house.

# That's why we think, pragmatically, it's very difficult to differentiate women and children as the bounder walker in their side of the house.
# That's why, Mr. Speaker, we think our side is even better for men, because total pile of AIDS will increase in our side of the house.
# And also, it is going to increase more accessibility for men, because PKO and people in PKO don't say, because you are men, you shouldn't receive this kind of medicine and this kind of support.

# Also, we told you that big narrative is not so important for militants there.
# Anyway, terrorists kill many people.
# Anyway, those kinds of conservative people will kill and rape women.
# But we told you, at least militants in legitimate entity are carrying it because they recognize a crime against humanity or those things.

# That's why, in a comparative manner, we think that we are trying to, we are successfully helping more women in the outside world.
# So, above all, their case might be standing in law school level.
# But after we explain the context and characterization of the situation of people there, their argument wasn't realistic.
# In most chaotic situations in the world, they don't care about those kinds of human rights.

# From their side of the house, they said the reason why women are less treated in the post-conflict society was that male victims weren't safe while women are already safe.
# And also that men feel they deserve reward or higher rights because they contribute to the ends of conflicts than women.
# But, Mr. Speaker, regardless of the existence of this narrative, usually women have relatively more difficulty in contributing to the conflict or humanitarian crisis in this military purpose of physical contribution.

# Because just because we remove those kinds of narratives, women are not suddenly going to participate in militaries or not trying to solve the humanitarian crisis in their side of the house.
# So, which is better?
# In their side, women are recognized as non-vulnerable and maybe sometimes violent, but they are not contributing to the conflict.
# But in our side of the house, women have justification not to contribute to the conflict because they are vulnerable, they are physically comparatively weaker.
# That's why they systematically couldn't contribute to the conflict.
# That's why, even though they didn't contribute to the conflict, they are not recognized as irresponsible actors or those kind of things.
# We told you how, right?

# And also they are saying that in post-conflict society, women are targeted on the street and women at all, not to wear slutty clothes and those kind of things.
# But we are not talking about those kinds of narratives.
# Because we are talking about narrative in a conflict, right?
# Maybe when it comes to real society, when it comes to safer places, there are different dominant narratives.
# Women can do anything that they want.
# We are talking about the narrative during the conflict situation.
# For that reason, we think even though logically there is some understanding, we think even though post-conflict female right, we are staying outside the house.

# ## Opposition 4th
# Two very simple questions. Is the dominance of the NIDILSC important and better assured?
# Second, could the dominance be toxic? So, from there, we told you that the dominance of the NIDILSC is important in three variety of ways.

# We firstly told you that it's better for the check and balance of the conflict zone, like we're able to establish the condition of rape in ICC.
# In similar ways, we have numerous check and balance institutions going on in conflicts and situations of humanitarian crisis.
# More pressure would lead to better institutions and check and balances of enforcement of those institutions.

# Their response from government was that, you know, it's probably a different reason, probably was male-centric and was able to tackle that.
# But they don't deny the fact that it was ultimately pressure that changed the situation.
# What we told you is not that the reason doesn't matter. We told you that this vulnerability leads to more social pressure to check on the situation of women, and that social pressure collectively moves the government, collectively moves international organisations, collectively moves the United Nations to change conditions.

# Second, we told you that it creates at least more aid. The pie is an exclusive thing, they get less.
# Their response is that, you know, children are enough to create sympathy, but also people don't care about the oily aid, which my partner dealt with.
# It's that the combination of children and women are the relative context, and that's why people are more able to sympathise with it.

# But they don't engage with the nuance of the fact that some people might sympathise with children, but you have to recognise some people also increasingly sympathise with women more than children.
# We are able to grab the minds of these people, which means that we are able to have more people driven to say that they want to send goods to these areas, they want to donate to these areas, and things like that.
# So children might be sufficient, but they miss out on important individuals and contexts, like feminists or other individuals.

# But also, see, it also lacks a nuance when we tell you that, you know, it's women that are particularly vulnerable because they are associated with sexual bullying, like rape, or gruesome attacks and things like that.
# That sort of narrative is impossible when you only use children.
# That is the most tragic and constant, detrimental situation that people don't even imagine, and that's why they want to help the situation.

# They are moved viscerally that they have to help the situation. That is why women are particularly an important tool in showing the vulnerability of the situation, and it simply does us very negative in the attempt to say that it's no different.
# Since they have a unique power, we get more power to provide aid and whether it's a better issue.
# But we also told you that it's important that we have a better understanding of the conflict situation.

# It's not that you need more firepower to protect against guerrillas, you need charity provisions, you need more protection about privacy, or better allocation of refugees to avoid rape between men attacking women and things like that.
# And their response was that, you know, it's the NGOs that have to understand it.
# But that's not the point. What we're telling you is that these NGOs also might be inexperienced or unaware of what is actually necessary.

# What is it compared to the substantiated harms that we provided?
# It's all about sexism, terrorism and things like that.
# We told you there are a variety of other reasons for that to happen. We don't understand why this scenario was particularly necessary and why it's the strongest thing.

# We also told you that particularly men feel responsibility and pride in the fact that they help women.
# It would be a shame if they say that women deserve to contribute less and it's okay that we feel wrong for women that they didn't contribute.
# They feel pride in the fact that they're helping women and going on with it. There was no response to that characterisation.

# But further, there was no response to us saying that this debate, as I said, is about westernised.
# We are not sure why this conflict on the humanitarian crisis area is looking at BBC or the exposure of Naira in the first place.
# That's why when they talk about the mindset being changed by the Naira about terrorists or people on the ground, they didn't actually prove the connection.

# ## Government 4th
# There was a news report showing the video of Syrian child who was drowning in the sea.
# That kind of video, it was sufficient and efficient enough in order to move public opinion toward acting against the refugee crisis.
# We believe those pictures or the video of a child or one woman or one man brutally being killed or brutally being injured is something efficient and sufficient enough in order to move public opinion.

# Their word of proof is that Y-dominant narrative of women being exposed in the westernized media is something important in order to gain money, in order to gain donation, in order to gain humanitarian aid, which was never proven from the side negative.
# We already told you from the deputy prime minister and government that that is not enough to win today's debate because this kind of dominant narrative means that BBC, CNN, those kinds of New York Times, they are always saying that women are being the target, women are being the victims, women being vulnerable, women being non-violent.
# That is the dominant narrative that they have to prove in today's debate.

# So we gave you two problems from the affirmative.
# Firstly, how the dominant narrative affects female's lives.
# We told you that they are used as a tool of the terrorist groups.

# Opposition told us that terrorists do not depend on such method or tactics.
# For example, they told us about the suicidal bombing or those kinds of things.
# Mr. Speaker, that is only limited to ISIS or those kinds of things.
# But we already have those kinds of Boko Haram using those kinds of female's lives in order to send those kinds of political messages.
# We believe those kinds of examples do not make them win this point.

# But secondly, we told you that the dominant narrative creates sexism in post-conflict society.
# Deputy leader of the opposition said that men do not do such kind of sexist actions because they have a duty to protect women.
# But Mr. Speaker from the leader of the opposition, he said that they are going to create another excuse in order to continue doing those kinds of sexist actions.
# Well, there is a contradiction in those kinds of characterizations of the conservative males.

# Therefore, Mr. Speaker, they could not prove why these post-conflict sexism do not happen in post-conflict society.
# They could not prove those kinds of points.
# Therefore, given that, opposition told us that the post-conflict sexism is much less important than immediate protection of female's lives.

# Well, Mr. Speaker, they are underestimating the power of sexism.
# It can lead to honor killing.
# It can lead to stoning.
# It can lead to the threat of female life, Mr. Speaker.
# We live in that kind of situation.
# That is not a mere dichotomy between lives and mere sexism.

# This kind of sexism cannot easily lead to the death or the threatening of female's life rights.
# We believe that is horrible, right?
# Therefore, Mr. Speaker, what we told you in today's debate is that, one, those kinds of post-conflict sexism is something horrible, something that can threaten female's lives.
# And second of all, we told you that the dominant narrative is affecting the current female's rights, the current female's lives.
# They are making terrorists attack females.
# They are making terrorists get the records or to get the opportunity to kill females, to rape females in the current status quo.
# We believe, what they told us, is that not mutually exclusive.
# Their benefit to gain money and gain more prize and allocate to females is not mutually exclusive.
# That can be done by children.
# That can be done by male victims who are brutally killed, brutally injured.
# We believe they cannot fulfill their burden of proof.
# We win this debate.

# # """

# text = "Who is shohei shotani?"

# # 一般的なmessageモデル
# response = client.responses.create(
#     model="gpt-4o",
#     input=text,
# )

# # 応答の表示
# current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-4]
# log_filename = f"logs/log_{current_time.replace(':', '-').replace(' ', '_').replace('.','-')}.txt"
# with open(log_filename, "a", encoding="utf-8") as log_file:
#     log_file.write(f"Timestamp: {current_time}\n")
#     log_file.write(f"Response detail: {response}")
#     log_file.write(f"Response:\n{response.output[0].content[0].text.strip()}\n")
#     log_file.write(f"Input Text:\n{text}\n")

# print(response.output[0].content[0].text.strip())

It seems like you might be referring to Shohei Ohtani. Shohei Ohtani is a Japanese professional baseball player known for his exceptional skills as both a pitcher and a hitter. He plays for the Los Angeles Angels in Major League Baseball (MLB). Ohtani is often praised for his rare ability to excel in both pitching and hitting, drawing comparisons to baseball legends like Babe Ruth. His versatility and talent have made him one of the most exciting players in the sport today.
